In [1]:
# downloading the tiny sh"akespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


--2024-04-27 12:37:19--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2024-04-27 12:37:19 (20.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
import torch
from torch import nn
from torch.nn import functional as F

In [6]:
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 2500 # changed from 5K to 10K
eval_interval = 500
learning_rate = 0.0001
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384 #changed from 384 to 512
n_head = 6 #changed from 6 to 12
n_layer = 6 #changed to 12 and then 12 to 6 Due to less compute power

In [12]:
import torch

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y



**AttentionHead**

In [13]:
class Head(nn.Module):

    def __init__(self):
        super().__init__()

        head_size = n_embd // n_head
        self.query  = nn.Linear(n_embd,head_size)
        self.key  = nn.Linear(n_embd,head_size)
        self.value  = nn.Linear(n_embd,head_size)
        self.tril = torch.tril(torch.ones(block_size,block_size))

    def forward(self,x):

        B,T,C = x.shape

        q = self.query(x)
        k = self.key(x)

        weights = q @ k.transpose(-2,-1)  * k.shape[-1]**-0.5
        weights = weights.masked_fill(self.tril[:T,:T].to('cuda:0') == 0, float("-inf")).to('cuda:0') #Removed .to('cuda:0')
        #weights = weights.masked_fill(self.tril[:T,:T]==0, float("-inf")).to('cuda:0')
        weights = F.softmax(weights,dim=-1)

        v = self.value(x)

        out = weights @ v

        return out



**MultiHeadAttention**

In [14]:
class MultiHeadAttention(nn.Module):

    def __init__(self,n_embd,n_head):
        super().__init__()

        head_size = n_embd // n_head
        self.heads = nn.ModuleList([Head() for _ in range(n_head)])
        self.proj = nn.Linear(head_size*n_head,n_embd)


    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out


**FeedForward Neural Network**

In [15]:
class FeedFoward(nn.Module):


    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
        )

    def forward(self, x):
        return self.net(x)

In [16]:
class DecoderBlock(nn.Module):

    def __init__(self,n_embd,n_head):
        super().__init__()

        self.mh_att = MultiHeadAttention(n_embd,n_head)
        self.nn = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)


    def forward(self,x):
        # x = self.mh_att(self.ln1(x))
        # x = self.nn(self.ln2(x))
        #Added the concept of Resnet architecture (as mentioned in the Atteniton all u need paper), which helps the model to remember the context of the previous tokens, so it could better generatlize the next token prediction.

        x_norm1= self.ln1(x)
        att = self.mh_att(x_norm1)
        x_norm2 = self.ln2(att)
        x = x + x_norm2
        x = self.nn(self.ln2(x))


        return x

**Language Model - GPT**

In [17]:
class LanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size,n_embd)
        self.pos_emb = nn.Embedding(block_size,n_embd)
        self.blocks = nn.Sequential(*[DecoderBlock(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.pred_head = nn.Linear(n_embd,vocab_size)

    def forward(self,idx,targets=None):

        token = self.token_emb(idx)
        pos = self.pos_emb(idx)

        x = token + pos

        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.pred_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [19]:
model = LanguageModel()
#model.load_state_dict(torch.load("model.pth"))
model = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) #trying different optimizers
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, momentum= 0.9) #changed the optimizer fromm AdamW to SGD and added momentum

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:

        out = {}
        perplexity = {}
        model.eval()
        for split in ['train', 'val']:
            losses = torch.zeros(eval_iters)
            for k in range(eval_iters):
                X, Y = get_batch(split)
                logits, loss = model(X, Y)
                losses[k] = loss.item()
            out[split] = losses.mean()
            perplexity[split] = torch.exp(losses.mean())
        model.train()
        print(f"step {iter}: train loss {out['train']:.4f}, val loss {out['val']:.4f}, train perplexity {perplexity['train']:.4f}, val perplexity {perplexity['val']}:.4f")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    torch.save(model.state_dict(), "model_ver_1.pth")

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

10.795841 M parameters
step 0: train loss 4.1313, val loss 4.1500, train perplexity 62.2568, val perplexity 63.43391418457031:.4f
step 500: train loss 3.3165, val loss 3.3538, train perplexity 27.5631, val perplexity 28.610515594482422:.4f
step 1000: train loss 3.3112, val loss 3.3511, train perplexity 27.4188, val perplexity 28.532913208007812:.4f
step 1500: train loss 3.3088, val loss 3.3531, train perplexity 27.3512, val perplexity 28.591333389282227:.4f
step 2000: train loss 3.3130, val loss 3.3489, train perplexity 27.4675, val perplexity 28.470735549926758:.4f
step 2499: train loss 3.3114, val loss 3.3452, train perplexity 27.4245, val perplexity 28.36541175842285:.4f

h3oAswpBwey t a uhv , as ihffyom e'
enHS V  vt? hoerdHr
bnoey nsiU,y,Lee oaoeSsaA eoyevgeawec ltslnrr
rtfg oos h- l
pyudNi
eoIsi,ethn olNtrde ds wlhnae:sll
 e oU,ythwr in
 nskdr.enfs,
oo ytatarl 
 tofihyrttii eioIdMRRefilstDtealso  cn m  sostron,
gitfshWtRcaenenpwr ongulormser h ds ,sfnaeoe:e aecT
ws mutH sd!aer e;